# Plot Radar Data

This notebook demonstrates plotting NEXRAD data using Python.

### Import Packages

In [ ]:
from datetime import datetime, timedelta

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
from metpy.calc import azimuth_range_to_lat_lon
from metpy.plots import colortables, USSTATES, USCOUNTIES
import numpy as np
from siphon.catalog import TDSCatalog
import xarray as xr

import warnings
warnings.filterwarnings("ignore")

### Helper Function

## Get Data

In [ ]:
date = datetime.utcnow()

# Create variables for URL generation
station = 'KLOT'

# Construct the data_url string
data_url = (f'https://thredds.ucar.edu/thredds/catalog/nexrad/level2/{station}/{date:%Y%m%d}/catalog.html')

# Get list of files available for particular day
cat = TDSCatalog(data_url)

# Grab dataset for desired time
dataset = cat.datasets.filter_time_nearest(date)

ds = xr.open_dataset(dataset.access_urls['OPENDAP'], decode_times=False,
                     decode_coords=False, mask_and_scale=True)

## Parse Data, Create Lat/Lon

In [ ]:
station = ds.Station
slat = ds.StationLatitude
slon = ds.StationLongitude
elevation = ds.StationElevationInMeters
vtime = datetime.strptime(ds.time_coverage_start, '%Y-%m-%dT%H:%M:%SZ')

sweep = 0
rng = ds.distanceR_HI.values
az = ds.azimuthR_HI.values[sweep]
ref = ds.Reflectivity_HI.values[sweep]

lon, lat = azimuth_range_to_lat_lon(az, rng, slon, slat)

## Make Plot

In [ ]:
cmap = colortables.get_colortable('NWSStormClearReflectivity')

fig, ax = plt.subplots(1, 1, figsize=(10, 9), subplot_kw=dict(projection=ccrs.PlateCarree()))

img = ax.pcolormesh(lon, lat, ref, vmin=-30, vmax=79, cmap=cmap)
plt.colorbar(img, aspect=50, pad=0)

ax.set_extent([-91, -85.5, 40, 43.5], ccrs.PlateCarree())
ax.set_aspect('equal', 'datalim')

ax.add_feature(USCOUNTIES.with_scale('5m'), edgecolor='darkgrey')
ax.add_feature(USSTATES.with_scale('5m'))

plt.title(f'{station}: {ds.Reflectivity_HI.name}', loc='left')
plt.title(f'Valid Time: {vtime}', loc='right')
plt.show()